# 전환율을 이용해서 실질월세 계산

In [2]:
import pandas as pd
from APICall import HouseType

2020년 데이터 수집 완료
2021년 데이터 수집 완료
2022년 데이터 수집 완료
2023년 데이터 수집 완료
2024년 데이터 수집 완료


In [4]:
def calc_newRent(houseType: HouseType, year: int) -> pd.DataFrame:
    FileName = f"data/{houseType.value}_{year}.csv"
    df = pd.read_csv(FileName)

    ChangeRateFileName = f"data/ChangeRate_SDMG_{houseType.value}.csv"
    ChangeRate = pd.read_csv(ChangeRateFileName)

    df["YM"] = df["dealDate"] // 100
    merged = pd.merge(df, ChangeRate, left_on="YM", right_on="Date", how="left")

    merged["monthlyRent"] = merged["monthlyRent"].astype(str).str.replace(',', '').astype(float)
    merged["deposit"] = merged["deposit"].astype(str).str.replace(',', '').astype(float)
    merged["calcedRent"] = merged["monthlyRent"] + merged["deposit"]*merged["ChangeRate"]/1200
    
    merged["RPA"] = merged["calcedRent"] / merged["excluUseAr"]
    merged["RPA"] = merged["RPA"].round(2)

    merged = merged.drop(columns=["YM", "Date"])
    columns = ["dealDate", "deposit", "monthlyRent", "ChangeRate", "calcedRent", "excluUseAr", "RPA"]
    for col in merged.columns:
        if col not in columns:
            columns.append(col)
    merged = merged[columns]
    print(merged)
    merged.to_csv("data/tmp.csv")
    

In [5]:
calc_newRent(HouseType.Apt, 2010)

       dealDate  deposit  monthlyRent  ChangeRate  calcedRent  excluUseAr  \
0      20110102  17000.0          0.0     7.20000  102.000000      59.788   
1      20110102  16000.0          0.0     7.20000   96.000000      60.060   
2      20110102  15000.0          0.0     7.20000   90.000000      59.700   
3      20110102  15000.0          0.0     7.20000   90.000000      59.760   
4      20110102  18000.0          0.0     7.20000  108.000000      84.960   
...         ...      ...          ...         ...         ...         ...   
34604  20191231  45000.0          0.0     3.71594  139.347750      59.790   
34605  20191231  53000.0          0.0     3.71594  164.120683      84.956   
34606  20191231  39000.0          0.0     3.71594  120.768050      84.210   
34607  20191231  37000.0          0.0     3.71594  114.574817      84.660   
34608  20191231  40000.0          0.0     3.71594  123.864667      84.780   

        RPA      aptNm  buildYear contractTerm contractType  floor  jibun  